In [1]:
import pandas as pd
import numpy as np

In [2]:
df_stt = pd.read_excel('data/고객입력정보_변환.xlsx', dtype={'세션아이디':str})
df_call = pd.read_excel('data/전체상담내역.xlsx', dtype={'세션 아이디':str})

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
df_stt.head()

,날짜,시간,전화번호,세션아이디,대화명,STT
0,2025-06-28,10:33:10,010-3068-4999,1751074360.2232152,세부 제품 확인 (공기청정기)_부정,아니요
1,2025-06-28,09:10:48,010-2778-9158,1751069426.968464,세부 제품 확인 (공기청정기)_부정,아니요 아 진짜
2,2025-06-27,15:07:44,010-8715-6221,1751004443.956842,세부 제품 확인 (공기청정기)_부정,아니요
3,2025-06-27,14:43:49,010-7140-6651,1751003008.2197189,세부 제품 확인 (공기청정기)_부정,아니요
4,2025-06-27,11:16:37,010-9355-6149,1750990575.2180952,세부 제품 확인 (공기청정기)_부정,아니요


In [4]:
df_call.head()

,날짜,시작시간,전화번호,세션 아이디,시나리오명,마지막 대화,상담시간,통화결과,이상키워드
0,2025-06-28,23:46:46,010-4566-2340,1751122005.2257622,하이텔레서비스(2차)_v1.4,문의확인 (2),00:00:28,상담 미완료,NaN
1,2025-06-28,23:43:38,010-6804-0345,1751121817.2257609,하이텔레서비스(2차)_v1.4,문의확인 (2),00:00:30,상담 미완료,NaN
2,2025-06-28,23:42:16,010-4566-2340,1751121736.2257605,하이텔레서비스(2차)_v1.4,문의확인 (2),00:00:46,상담 미완료,NaN
3,2025-06-28,23:39:18,010-3656-7590,1751121558.2257601,하이텔레서비스(2차)_v1.4,문의 확인,00:00:25,상담 미완료,NaN
4,2025-06-28,23:39:15,010-4566-2340,1751121554.2257599,하이텔레서비스(2차)_v1.4,문의확인 (2),00:00:43,상담 미완료,NaN


In [5]:
df_stt.rename(columns={"세션아이디":"session_id", "전화번호":"phone_number"}, inplace=True)
df_call.rename(columns={"세션 아이디":"session_id", "전화번호":"phone_number"}, inplace=True)

In [6]:
# df_call.sort_values(by='시작시간', ascending=False, inplace=True)
# df_stt['시작시간'] = df_stt.apply(lambda x:df_call.loc[(df_call['전화번호']==x['전화번호']) & (df_call['시작시간']<=x['시간']), '시작시간'].iloc[0], axis=1)

In [7]:
df = df_stt.merge(df_call, how='left', left_on=['session_id', 'phone_number', '날짜'], right_on=['session_id', 'phone_number', '날짜'])
df.head()

,날짜,시간,phone_number,session_id,대화명,STT,시작시간,시나리오명,마지막 대화,상담시간,통화결과,이상키워드
0,2025-06-28,10:33:10,010-3068-4999,1751074360.2232152,세부 제품 확인 (공기청정기)_부정,아니요,10:32:40,하이텔레서비스(2차)_v1.4,증상 확인,00:01:18,상담 미완료,NaN
1,2025-06-28,09:10:48,010-2778-9158,1751069426.968464,세부 제품 확인 (공기청정기)_부정,아니요 아 진짜,09:10:27,하이텔레서비스(2차)_v1.4,증상 확인,00:00:56,상담 미완료,NaN
2,2025-06-27,15:07:44,010-8715-6221,1751004443.956842,세부 제품 확인 (공기청정기)_부정,아니요,15:07:23,하이텔레서비스(2차)_v1.4,증상 확인,00:01:12,상담 미완료,NaN
3,2025-06-27,14:43:49,010-7140-6651,1751003008.2197189,세부 제품 확인 (공기청정기)_부정,아니요,14:43:29,하이텔레서비스(2차)_v1.4,증상 확인,00:00:53,상담 미완료,NaN
4,2025-06-27,11:16:37,010-9355-6149,1750990575.2180952,세부 제품 확인 (공기청정기)_부정,아니요,11:16:16,하이텔레서비스(2차)_v1.4,증상 확인,00:00:43,상담 미완료,NaN


In [8]:
df_stt.shape

(6970, 6)

In [9]:
df.drop_duplicates(inplace=True)
df.shape

(6970, 12)

In [10]:
# 콜 넘버링하기
# df.sort_values(['시간', 'session_id'], inplace=True)
# idarr = df.session_id.unique()
# df["No"] = df.session_id.apply(lambda x:int(np.where(idarr==x)[0])+1)

# df

In [11]:
# 발화턴 추가
df.sort_values(['날짜', '시간', 'session_id'], inplace=True)

turn = []
sid_b = 0

for sid in df.session_id:
    if sid == sid_b:
        n += 1
    else:
        n = 1
    turn.append(n)
    sid_b = sid

df['발화턴'] = turn
df

,날짜,시간,phone_number,session_id,대화명,STT,시작시간,시나리오명,마지막 대화,상담시간,통화결과,이상키워드,발화턴
3935,2025-06-26,05:45:42,010-9820-6654,1750884317.913289,이전설치,네 설치,05:45:18,하이텔레서비스(2차)_v1.4,이전설치 홈페이지연결 여부,00:00:52,상담 미완료,NaN,1
2640,2025-06-26,05:48:34,010-6489-9705,1750884493.913292,상담사 연결 요청,에어컨 상담사 연결해줘,05:48:13,하이텔레서비스(2차)_v1.4,문의 확인,00:00:42,상담 미완료,NaN,1
4857,2025-06-26,05:49:02,010-6224-3569,1750884512.913294,구매,구매 역,05:48:33,하이텔레서비스(2차)_v1.4,구매 질의,00:01:06,상담 미완료,NaN,1
3242,2025-06-26,06:15:35,010-4022-0753,1750886114.2099098,제품 대분류 (냉장고),네 냉장고,06:15:14,하이텔레서비스(2차)_v1.4,세부 제품 확인 (냉장고),00:00:47,상담 미완료,NaN,1
5079,2025-06-26,06:26:37,010-4019-6821,1750886767.2099126,세부 제품 확인 (김치냉장고)_부정,아니요,06:26:08,하이텔레서비스(2차)_v1.4,증상 확인,00:01:35,상담 미완료,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6298,2025-06-28,22:43:39,010-8470-5153,1751118198.982203,증상 1,네 실외기가 안 돌아가요,22:43:19,하이텔레서비스(2차)_v1.4,질의 2,00:01:33,상담 미완료,NaN,4
2280,2025-06-28,23:25:24,010-7340-9829,1751120702.2257570,제품 소분류 (노트북),네 노트북,23:25:03,하이텔레서비스(2차)_v1.4,증상 확인,00:00:48,상담 미완료,NaN,1
340,2025-06-28,23:26:06,010-5098-4931,1751120745.982277,제품 대분류 (에어컨),네 에어컨,23:25:45,하이텔레서비스(2차)_v1.4,질의 1,00:01:19,상담 미완료,NaN,1
3515,2025-06-28,23:26:06,010-5098-4931,1751120745.982277,세부 제품 확인 (에어컨)_천장형,천장형 에어컨,23:25:45,하이텔레서비스(2차)_v1.4,질의 1,00:01:19,상담 미완료,NaN,2


In [12]:
df = df[['session_id', 'phone_number', '날짜', '시간', '마지막 대화', '상담시간', '통화결과', '대화명', 'STT']]
df




,session_id,phone_number,날짜,시간,마지막 대화,상담시간,통화결과,대화명,STT
3935,1750884317.913289,010-9820-6654,2025-06-26,05:45:42,이전설치 홈페이지연결 여부,00:00:52,상담 미완료,이전설치,네 설치
2640,1750884493.913292,010-6489-9705,2025-06-26,05:48:34,문의 확인,00:00:42,상담 미완료,상담사 연결 요청,에어컨 상담사 연결해줘
4857,1750884512.913294,010-6224-3569,2025-06-26,05:49:02,구매 질의,00:01:06,상담 미완료,구매,구매 역
3242,1750886114.2099098,010-4022-0753,2025-06-26,06:15:35,세부 제품 확인 (냉장고),00:00:47,상담 미완료,제품 대분류 (냉장고),네 냉장고
5079,1750886767.2099126,010-4019-6821,2025-06-26,06:26:37,증상 확인,00:01:35,상담 미완료,세부 제품 확인 (김치냉장고)_부정,아니요
...,...,...,...,...,...,...,...,...,...
6298,1751118198.982203,010-8470-5153,2025-06-28,22:43:39,질의 2,00:01:33,상담 미완료,증상 1,네 실외기가 안 돌아가요
2280,1751120702.2257570,010-7340-9829,2025-06-28,23:25:24,증상 확인,00:00:48,상담 미완료,제품 소분류 (노트북),네 노트북
340,1751120745.982277,010-5098-4931,2025-06-28,23:26:06,질의 1,00:01:19,상담 미완료,제품 대분류 (에어컨),네 에어컨
3515,1751120745.982277,010-5098-4931,2025-06-28,23:26:06,질의 1,00:01:19,상담 미완료,세부 제품 확인 (에어컨)_천장형,천장형 에어컨


In [13]:
air_ids = df.loc[df['대화명'].str.contains("에어컨"), "session_id"].unique()
df.loc[df["session_id"].isin(air_ids), "제품"] = "에어컨"

ref_ids = df.loc[df['대화명'].str.contains("냉장고"), "session_id"].unique()
df.loc[df["session_id"].isin(ref_ids), "제품"] = "냉장고"

C:\Users\Admin\AppData\Local\Temp\ipykernel_25216\574977006.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df["session_id"].isin(air_ids), "제품"] = "에어컨"


In [14]:
df.loc[~df['제품'].isna(), '마지막 대화'].unique()

array(['세부 제품 확인 (냉장고)', '증상 확인', '질의 1', 'R - 진행', '세부 제품 확인 (에어컨)',
       '질의 2', '케어십해지여부', '문의 질의', '세부 제품 확인 (김치냉장고)', '자가해결 유도'],
      dtype=object)

In [15]:
lasts = ['R - 진행', '질의 1', '질의 2', '증상 확인', '세부 제품 확인 (김치냉장고)', '세부 제품 확인 (냉장고)', '세부 제품 확인 (에어컨)']
df.loc[(~df['제품'].isna()) & (df['마지막 대화'].isin(lasts)), "분석대상"] = "Y"

C:\Users\Admin\AppData\Local\Temp\ipykernel_25216\414604422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[(~df['제품'].isna()) & (df['마지막 대화'].isin(lasts)), "분석대상"] = "Y"


In [16]:
df.fillna("", inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_25216\2065217620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna("", inplace=True)


In [17]:
df

,session_id,phone_number,날짜,시간,마지막 대화,상담시간,통화결과,대화명,STT,제품,분석대상
3935,1750884317.913289,010-9820-6654,2025-06-26,05:45:42,이전설치 홈페이지연결 여부,00:00:52,상담 미완료,이전설치,네 설치,,
2640,1750884493.913292,010-6489-9705,2025-06-26,05:48:34,문의 확인,00:00:42,상담 미완료,상담사 연결 요청,에어컨 상담사 연결해줘,,
4857,1750884512.913294,010-6224-3569,2025-06-26,05:49:02,구매 질의,00:01:06,상담 미완료,구매,구매 역,,
3242,1750886114.2099098,010-4022-0753,2025-06-26,06:15:35,세부 제품 확인 (냉장고),00:00:47,상담 미완료,제품 대분류 (냉장고),네 냉장고,냉장고,Y
5079,1750886767.2099126,010-4019-6821,2025-06-26,06:26:37,증상 확인,00:01:35,상담 미완료,세부 제품 확인 (김치냉장고)_부정,아니요,냉장고,Y
...,...,...,...,...,...,...,...,...,...,...,...
6298,1751118198.982203,010-8470-5153,2025-06-28,22:43:39,질의 2,00:01:33,상담 미완료,증상 1,네 실외기가 안 돌아가요,에어컨,Y
2280,1751120702.2257570,010-7340-9829,2025-06-28,23:25:24,증상 확인,00:00:48,상담 미완료,제품 소분류 (노트북),네 노트북,,
340,1751120745.982277,010-5098-4931,2025-06-28,23:26:06,질의 1,00:01:19,상담 미완료,제품 대분류 (에어컨),네 에어컨,에어컨,Y
3515,1751120745.982277,010-5098-4931,2025-06-28,23:26:06,질의 1,00:01:19,상담 미완료,세부 제품 확인 (에어컨)_천장형,천장형 에어컨,에어컨,Y


In [18]:
yymmdd = lambda x:x[2:4]+x[5:7]+x[8:10]
df.to_excel(f"result/voicebot_stt_{yymmdd(df['날짜'].min())}~{yymmdd(df['날짜'].max())}.xlsx", index=False)